In [18]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import sys
import os

In [19]:
img_width, img_height = 150, 150
 
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 200
nb_validation_samples = 40
nb_epoch = 10

In [20]:
# VGG16の既存の１０００クラスの出力を使わないため、　include_top=False　として出力層を含まない状態でロードする。
# VGG16　は本来、１つの入力画像に対して１０００クラス分の確率を出力するような構造になっている。
model = VGG16(include_top=False ,weights='imagenet')

In [21]:
# model　のsummary　を確認。　出力層が含まれていない。
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [22]:
# ImageDataGenerator() 学習用画像をロードするためのジェネレータを生成する。スケール変換を指定。
import keras

datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [23]:
# flow_from_directory() 学習時や予測時に、指定されたディレクトリから、batch_sizeの数だけ画像を読み込んで
#１ミニバッチ分の画像と正解ラベルを返してくれるイテレータを生成できる。
itr_train = datagen.flow_from_directory(
                train_data_dir,
                target_size=(img_width, img_height),
                batch_size=32,
                class_mode=None,
                shuffle=False)

itr_validation = datagen.flow_from_directory(
            validation_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)

# predict_generator() ジェネレータから生成されたデータに対しての予測をする。
features_train = model.predict_generator(itr_train, nb_train_samples)
# np.save(file, arr) file データを保存する先を指定する。保存する配列。array_like配列に相当するオブジェクト
np.save(open('features_train.npy', 'wb'), features_train)

features_validation = model.predict_generator(itr_validation, nb_validation_samples)
np.save(open('features_validation.npy', 'wb'), features_validation)

Found 200 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [25]:
features_validation.shape

(40, 4, 4, 512)

In [11]:
np.array([0] * int(nb_train_samples / 2) + [1])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [13]:
np.array([1] * int(nb_train_samples / 2))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout

def train_top_model():
    features_train=np.load(open('features_train.npy','rb'))
    # traiデータにおける犬と猫の正解ラベルを作成。[0] が１００。[1]が１００。
    train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))
    features_validation = np.load(open('features_validation.npy','rb'))
    validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))
     
    # VGG１６に追加する出力層の部分を構築。
    top_model = Sequential()
    top_model.add(Flatten(input_shape=features_validation.shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
 
    top_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
 
    top_model.fit(features_train, train_labels,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(features_validation, validation_labels))
 
 
train_top_model()

/Users/takahashikoji/.pyenv/versions/anaconda3-4.3.0/envs/seq2seq/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 200 samples, validate on 40 samples
Epoch 1/10
200/200 [==============================] - 1s 3ms/step - loss: 5.9021 - acc: 0.4900 - val_loss: 0.8356 - val_acc: 0.5250
Epoch 2/10
200/200 [==============================] - 0s 1ms/step - loss: 1.1567 - acc: 0.6100 - val_loss: 0.4711 - val_acc: 0.8250
Epoch 3/10
200/200 [==============================] - 0s 1ms/step - loss: 0.6988 - acc: 0.7150 - val_loss: 0.9184 - val_acc: 0.6000
Epoch 4/10
200/200 [==============================] - 0s 1ms/step - loss: 0.3804 - acc: 0.8350 - val_loss: 0.4361 - val_acc: 0.8250
Epoch 5/10
200/200 [==============================] - 0s 1ms/step - loss: 0.5669 - acc: 0.7600 - val_loss: 0.2459 - val_acc: 0.9250
Epoch 6/10
200/200 [==============================] - 0s 1ms/step - loss: 0.1844 - acc: 0.9350 - val_loss: 0.2876 - val_acc: 0.9250
Epoch 7/10
200/200 [==============================] - 0s 1ms/step - loss: 0.2601 - acc: 0.8900 - val_loss: 0.2327 - val_acc: 0.9000
Epoch 8/10
200/200 [===========